In [24]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

In [25]:
import os
import sys
import pandas as pd
import csv
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [26]:
from sklearn.linear_model import Ridge,LogisticRegression

# stack_clf = LogisticRegression(n_jobs=-1)

stack_clf = Ridge(alpha=1.0)

In [27]:
from xgboost import XGBRegressor,plot_importance
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import Normalizer
import xgboost as xgb

In [28]:
from matplotlib import pyplot
%matplotlib inline

In [29]:
from mlxtend.classifier import StackingClassifier

In [30]:
data_folder = '../data'
data_files = os.listdir(data_folder)

In [31]:
year_2014 = pd.read_csv(data_folder+'/'+data_files[0])
year_2014.head()

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,...,Wind Direction Units,Wind Speed,Snow Depth Units,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Version
0,SUNY,22306,-,-,India,24.65,73.75,5.5,0,5.5,...,Degrees,m/s,m,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,unknown
1,Year,Month,Day,Hour,Minute,DHI,DNI,GHI,Clearsky DHI,Clearsky DNI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1,1,0,30,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,1,1,1,30,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,1,1,2,30,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
new_columns = year_2014.iloc[1]
year_all = year_2014[2:]
year_all.columns =new_columns
year = year_all.loc[:, year_all.columns.notnull()]

# for counter,i in enumerate(new_columns):
#     print(i)


year = year.drop(columns=['Year','Month','Day','Minute','DHI','DNI','Clearsky DHI','Clearsky DNI','Fill Flag','Dew Point','Clearsky GHI','Solar Zenith Angle','Wind Direction','Snow Depth','Precipitable Water'])

In [33]:
year = year.astype('float64')
year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 2 to 8761
Data columns (total 6 columns):
Hour                 8760 non-null float64
GHI                  8760 non-null float64
Temperature          8760 non-null float64
Pressure             8760 non-null float64
Relative Humidity    8760 non-null float64
Wind Speed           8760 non-null float64
dtypes: float64(6)
memory usage: 410.7 KB


In [34]:
X = year.drop(columns=['GHI'])
Y = year.pop('GHI')

# transformer = Normalizer().fit(X)
# X_norm = transformer.transform(X)


# print(transformer)
# print(X_norm)

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

### models 

In [35]:
xgb_1 = XGBRegressor(learning_rate=0.2,max_depth=8,n_estimators=20)
xgb_2 = XGBRegressor(learning_rate=0.2,max_depth=6,n_estimators=40)
xgb_3 = XGBRegressor(learning_rate=0.2,max_depth=5,n_estimators=80)
xgb_4 = XGBRegressor(learning_rate=0.2,max_depth=5,n_estimators=100)
xgb_5 = XGBRegressor(learning_rate=0.2,max_depth=6,n_estimators=80)
xgb_6 = XGBRegressor(learning_rate=0.2,max_depth=6,n_estimators=100)

In [36]:
sclf = StackingClassifier(classifiers=[xgb_1, xgb_2, xgb_3,xgb_4,xgb_5,xgb_6], 
                          meta_classifier=stack_clf)

In [37]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [38]:
import math
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
for clf, label in zip([xgb_1, xgb_2, xgb_3,xgb_4,xgb_5,xgb_6,sclf], 
                      ['xgb_1', 
                       'xgb_2', 
                       'xgb_3',
                       'xgb_4',
                       'xgb_5', 
                       'xgb_6', 
                       'Forest']):

    scores = sklearn.model_selection.cross_val_score(clf, X, Y, 
                                              cv=kfold, scoring='neg_mean_squared_error')
    print("RSME: %0.2f (+/- %0.2f) [%s]" 
          % (math.sqrt(-scores.mean()), scores.std(), label))

/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/pyth

RSME: 61.62 (+/- 2828.95) [xgb_1]


/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/pyth

RSME: 60.99 (+/- 2807.74) [xgb_2]


/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/pyth

...

RSME: 60.99 (+/- 2793.04) [xgb_5]


/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/pyth

RSME: 61.11 (+/- 2788.97) [xgb_6]


/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/pyth

RSME: 63.41 (+/- 2831.21) [Forest]


In [39]:
param_grid = {'meta-ridge__alpha': [1.0]}

grid = GridSearchCV(estimator=sclf, 
                    param_grid=param_grid,
                    cv=kfold,
                    scoring="neg_mean_squared_error",
                    refit=True)
grid.fit(X, Y)

cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r" %(math.sqrt(-grid.cv_results_[cv_keys[0]][r]), grid.cv_results_[cv_keys[1]][r] / 2.0,grid.cv_results_[cv_keys[2]][r]))

print('Best parameters: %s' % grid.best_params_)
print('Accuracy: %.2f' % grid.best_score_)

/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/primus/durgeshapi/ensemble/env/lib/pyth

71.338 +/- 1389.62 {'meta-ridge__alpha': 1.0}
Best parameters: {'meta-ridge__alpha': 1.0}
Accuracy: -5089.17


In [64]:
sclf.get_params().keys()

dict_keys(['xgbregressor-1', 'xgbregressor-2', 'xgbregressor-3', 'xgbregressor-4', 'xgbregressor-5', 'xgbregressor-6', 'xgbregressor-1__base_score', 'xgbregressor-1__booster', 'xgbregressor-1__colsample_bylevel', 'xgbregressor-1__colsample_bytree', 'xgbregressor-1__gamma', 'xgbregressor-1__importance_type', 'xgbregressor-1__learning_rate', 'xgbregressor-1__max_delta_step', 'xgbregressor-1__max_depth', 'xgbregressor-1__min_child_weight', 'xgbregressor-1__missing', 'xgbregressor-1__n_estimators', 'xgbregressor-1__n_jobs', 'xgbregressor-1__nthread', 'xgbregressor-1__objective', 'xgbregressor-1__random_state', 'xgbregressor-1__reg_alpha', 'xgbregressor-1__reg_lambda', 'xgbregressor-1__scale_pos_weight', 'xgbregressor-1__seed', 'xgbregressor-1__silent', 'xgbregressor-1__subsample', 'xgbregressor-2__base_score', 'xgbregressor-2__booster', 'xgbregressor-2__colsample_bylevel', 'xgbregressor-2__colsample_bytree', 'xgbregressor-2__gamma', 'xgbregressor-2__importance_type', 'xgbregressor-2__learn